In [3]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
from enums import Universe

### Test

In [2]:
import pandas as pd
import warnings
from portfolio import FractilePortfolio
import plotly.graph_objects as go
from Utilities import Universe
warnings.filterwarnings("ignore")


# df_factors = pd.read_excel("Input/S&P500_Factors.xlsx")
# df_factors = df_factors.rename(columns={"Value (P/B)": "Value", 'Momentum (12m)': "Momentum",
#                                         "Low Volatility (252d)": "Low Vol", 'Quality (ROE)': "Quality"})

from analysis import PortfolioAnalysis
ptf_analysis = PortfolioAnalysis(universe = Universe.SP500)
sensibilities = ptf_analysis.get_factor_information("Momentum",["Momentum","Value","Quality","Low Volatility"],"2017-01-01")

INFO:root:Chargement de l'univers...
INFO:root:Données chargées et triées par date.


Nombre de tickers sélectionnés : 505
Nombre de tickers communs dans toutes les données : 389
Période d'analyse : 2015-12-31 00:00:00 -> 2020-01-03 00:00:00
Calcul du facteur Value (P/B)...
Calcul du facteur Momentum (12-1 month return)...
Calcul du facteur Quality (ROE)...
Calcul du facteur Low Volatility (252-day vol)...
Calcul des facteurs terminé.


In [2]:
sensibilities

,Dates,Momentum,Value,Quality,Low Volatility
0,2017-01-03,2.343626,-0.145591,-0.107964,0.207201
1,2017-01-04,2.326517,-0.145591,-0.107964,0.206615
2,2017-01-05,2.273308,-0.145591,-0.107964,0.194787
3,2017-01-06,2.239111,-0.145591,-0.107964,0.187755
4,2017-01-09,2.216368,-0.145591,-0.107964,0.190837
...,...,...,...,...,...
749,2019-12-24,0.161221,-0.111745,-0.060272,0.008221
750,2019-12-26,0.139258,-0.111745,-0.060272,-0.002996
751,2019-12-27,0.100986,-0.111745,-0.060272,-0.003332
752,2019-12-30,0.115690,-0.111745,-0.060272,-0.006478


In [4]:
df = sensibilities.copy()

In [8]:
df["Dates"] = pd.to_datetime(df["Dates"])  
start_date = df["Dates"].iloc[0]  # Première date du dataset

# Calcul du nombre de mois écoulés
df["Months_Since_Start"] = (df["Dates"] - start_date) / pd.Timedelta(days=30)

# Création du graphique interactif avec Plotly
fig = px.line(df, x="Months_Since_Start", y=df.columns[1:-1], 
              labels={"value": "Sensibilité", "Months_Since_Start": "Nombre de mois depuis la date principale"},
              title="Évolution des sensibilités des facteurs")

# Affichage du graphique
fig.show()

## Preprocessing

In [4]:
universe  = Universe.SP500
compo_universe = pd.read_parquet(f"Input_parquet/{universe.value['compo']}")
universe_prices = pd.read_parquet(f"Input_parquet/{universe.value['Price']}")
universe_roe = pd.read_parquet(f"Input_parquet/{universe.value['ROE']}")
universe_ptb = pd.read_parquet(f"Input_parquet/{universe.value['Price To Book']}")

In [5]:
universe_prices = universe_prices.rename(columns={"DATE":"Date"})

In [10]:
universe_prices['Date'].dtypes

dtype('<M8[ns]')

In [6]:
len(universe_prices)

5761

In [9]:
df_roe = pd.read_excel("Input/ROE.xlsx") 
df_ptb = pd.read_excel("Input/PTB.xlsx")

In [22]:
df_roe = df_roe.rename(columns={"Unnamed: 0":"Date"})
df_ptb = df_ptb.rename(columns={"Unnamed: 0":"Date"})

In [25]:
dates_communes = set(universe_prices['Date']).intersection(df_roe['Date']).intersection(df_ptb['Date'])

In [26]:
len(dates_communes)

5761

In [27]:
df_roe_filtered = df_roe[df_roe['Date'].isin(dates_communes)]
df_ptb_filtered = df_ptb[df_ptb['Date'].isin(dates_communes)]

In [33]:
df_roe_filtered.reset_index(drop=True, inplace=True)
df_ptb_filtered.reset_index(drop=True, inplace=True)

In [17]:
compo_universe.to_parquet("Input_parquet/COMPO_SP500.parquet", engine='pyarrow', compression='snappy')

In [10]:
universe_prices.to_parquet("Input_parquet/PX_LAST.parquet", engine='pyarrow', compression='snappy')

In [34]:
df_roe_filtered.to_parquet("Input_parquet/ROE.parquet", engine='pyarrow', compression='snappy')

In [35]:
df_ptb_filtered.to_parquet("Input_parquet/PTB.parquet", engine='pyarrow', compression='snappy')

In [5]:
computation_date_str = "2010-01-01"

In [15]:
computation_date_dt = datetime.strptime(computation_date_str, "%Y-%m-%d")
closest_row_universe = compo_universe[compo_universe['Date'] <= computation_date_dt].sort_values('Date').tail(1)

if not closest_row_universe.empty:
    # Prendre uniquement les colonnes où la valeur est égale à 1
    closest_date = closest_row_universe['Date'].iloc[0]
    tickers_universe = closest_row_universe.loc[:, closest_row_universe.eq(1).iloc[0]].columns.to_list()
    print(len(tickers_universe))
else:
    print("Aucune ligne correspondant à la date trouvée.")

500


In [9]:
tickers_universe = compo_universe.loc[compo_universe['Date']==closest_date, compo_universe.eq(1).any()]

In [10]:
tickers_universe

,0111145D UN,0202445Q UN,0203524D UN,0226226D UN,0544749D UN,0772031D UN,0848680D UN,0910150D US,0948669D UN,0961514D UN,...,TEL UN,BLK UN,PLTR UW,TPL UN,APO UN,APTV UN,LII UN,WDAY UW,DPZ UW,MRP UN
95,1,1,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Old 

In [ ]:
# def fetch_equity_factors(tickers):
#     data = []
#     for ticker in tickers:
#         stock = yf.Ticker(ticker)
#
#         try:
#             # Price-to-Book Ratio (Value)
#             pb_ratio = stock.info.get('priceToBook', None)
#
#             # 12-month Momentum (Momentum)
#             hist = stock.history(period='1y')
#             if not hist.empty:
#                 momentum = (hist['Close'][-1] / hist['Close'][0]) - 1
#             else:
#                 momentum = None
#
#             # Return on Equity (Quality)
#             roe = stock.info.get('returnOnEquity', None)
#
#             # Percentage Growth in Total Assets (Investment)
#             total_assets = stock.balance_sheet.loc['Total Assets'] if 'Total Assets' in stock.balance_sheet else None
#             if total_assets is not None and len(total_assets) > 1:
#                 investment = (total_assets[-1] - total_assets[-2]) / total_assets[-2]
#             else:
#                 investment = None
#
#             # 252-day return volatility (Low Volatility)
#             if not hist.empty:
#                 daily_returns = hist['Close'].pct_change().dropna()
#                 low_volatility = daily_returns.std() * (252 ** 0.5)
#             else:
#                 low_volatility = None
#
#             # Append to data
#             data.append({
#                 'Ticker': ticker,
#                 'Value (P/B)': pb_ratio,
#                 'Momentum': momentum,
#                 'Quality (ROE)': roe,
#                 'Investment (Asset Growth)': investment,
#                 'Low Volatility (252d)': low_volatility,
#             })
#
#         except Exception as e:
#             print(f"Error fetching data for {ticker}: {e}")
#
#     return pd.DataFrame(data)
#
# df = pd.read_json("sp500.json")
# tickers = df["Ticker"].tolist()
# df_infos = fetch_equity_factors(tickers)
# print(df_infos.head())
# df_infos.to_csv("sp500_factors.csv", index=False)


## Retrieve Dates

In [ ]:
df = pd.read_excel("Input/PX_LAST.xlsx")
df_ref = pd.read_excel("Input/df_ref.xlsx")

In [4]:
def find_unique_columns_with_progress(df):
    different_columns = []
    # Utiliser tqdm pour suivre la progression des colonnes
    for col in tqdm(df.columns, desc="Analyse des colonnes"):
        # Comparaison avec la valeur suivante
        differences = df[col] != df[col].shift(-1)
        # Si toutes les différences sont True, ajouter la colonne à la liste
        if differences[:-1].all():
            different_columns.append(col)
    
    return different_columns

# Appel de la fonction
result = find_unique_columns_with_progress(df)
print("Colonnes avec des valeurs différentes de la suivante :", result)

Analyse des colonnes:   0%|          | 0/856 [00:00<?, ?it/s]

Analyse des colonnes: 100%|██████████| 856/856 [00:00<00:00, 3558.59it/s]

Colonnes avec des valeurs différentes de la suivante : []


In [5]:
start_date = pd.to_datetime("2002-01-31")
end_date = pd.to_datetime("2025-02-15")

# Générer une plage de 6002 dates uniformément espacées
dates = pd.date_range(start=start_date, end=end_date, freq='B')

In [6]:
differences = df["NDSN UW Equity"] != df["NDSN UW Equity"].shift(-1)

In [7]:
differences_ref = df_ref["NDSN UW Equity"] != df_ref["NDSN UW Equity"].shift(-1)

In [8]:
index_df = differences[differences].index
index_ref = differences_ref[differences_ref].index

In [9]:
df_ref_fin = df_ref.iloc[index_ref].reset_index(drop=True)
df_fin = df.iloc[index_df].reset_index(drop=True)

In [10]:
df_fin.insert(0, 'DATE',df_ref_fin['Date'])

In [11]:
df_fin.to_excel("Input/PX_LAST_REAL.xlsx", index = False)